In [24]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [25]:
import subprocess
import astropy.table
import astropy.io.fits as fits
import numpy as np
from copy import deepcopy
import random
from astropy.table import Table
import os 
import matplotlib.pyplot as plt 
import fitsio 
import collections

In [26]:
#directories that would be using 
WLD = '/Users/Ismael/code/lensing/WeakLensingDeblending/'
aegis = '/Users/Ismael/aegis/data/'
SEx = '/Users/Ismael/aegis/data/sextractor_runs/'
aegis_slac = '/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data'
temp_data = '/Users/Ismael/temp_data'
os.chdir(WLD)
import descwl

In [27]:
#inputs, put all input files in a dict, 
inputs = dict(
input_name = '/Users/Ismael/aegis/data/intermediate_fits/section001.fits',
config_file = '/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/default.sex',
image_fits = '/Users/Ismael/aegis/data/sextractor_runs/image1.fits',
image_fits_slac = '/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/image1.fits',
)

In [28]:
#convert inputs to location in temp
temps = dict()
for f in inputs: 
    l = inputs[f].split("/")
    temp_file = '{0}{1}'.format('/Users/Ismael/temp_data/',l[-1])
    temps[f] = temp_file

In [57]:
parameter_names = ['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']

runs = collections.OrderedDict()
runs['default']= [1.5,5,'default.conv']
runs['point_optimized']=[5.0,1.,'gauss_3.5_9x9.conv']
runs['extended_optimized']=[1.5,5,'gauss_2.0_5x5.conv']
runs['extended_optimized_1']=[1.5,5,'gauss_3.5_9x9.conv']
runs['extended_optimized_2']=[1.5,1.,'gauss_2.0_5x5.conv']
runs['extended_optimized_3']=[5.0,5,'gauss_2.0_5x5.conv']


output_names = {}
output_names_slac = {}
final_fits_names = collections.OrderedDict()

for key in runs: 
    output_names[key] = '/Users/Ismael/temp_data/out_{0}.cat'.format(key)
    output_names_slac[key]='/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/out_{0}.cat'.format(key)
    final_fits_names[key] = '/Users/Ismael/temp_data/{0}_results.fits'.format(key)

In [45]:
use = [0,0,0,0,0,0] #one for each run in runs, 1 means it will be ran. 

In [46]:
#algorithm for ambiguous blends - returns indices of ambiguously blended objects in terms of the table of full catalogue. 
#ambiguously blended means that at least one non-detected true object is less than a unit of effective distance away from an ambiguous object. 
def detected_ambiguous_blends(table,matched_indices,detected):
    """
    table: table containing entries of all galaxies of the catalog. 
    matched_indices: indices of galaxies in table that were detected by SExtractor (primary  matched)
    detected: Table of detected objects by SExtractor which also contains their information as measured by SExtractor. 
    """
    ambiguous_blends_indices = []
    for index,gal_row in enumerate(table):
        if gal_row['match'] == -1: #loop through true not primarily matched objects. 
            
            #calculate of effective distances of true non-primary match object with all primarily matched objects. 
            effective_distances = list(np.sqrt((detected['X_IMAGE']-gal_row['dx'])**2 + (detected['Y_IMAGE']-gal_row['dy'])**2)/(detected['SIGMA'] + gal_row['psf_sigm']))
            
            #mark all objects with effective distance <1. as ambiguosly blended. 
            marked_indices = [matched_indices[i] for i,distance in enumerate(effective_distances) if distance < 1.]
            
            #if at least one index was marked as ambiguous, add both the primarily match object and the true non-primary object as ambiguous. 
            if len(marked_indices) > 0:
                ambiguous_blends_indices.append(index)
                ambiguous_blends_indices.extend(marked_indices)
    return set(ambiguous_blends_indices)

In [47]:
#some interesting subsets of the simulation
iso_gal = lambda cat: cat[cat['purity'] > .98] #isolated galaxies
grp_gal = lambda cat: cat[cat['purity'] <= .98] #galaxies in a group of 2 or more. 

#'good' galaxies satisfy the reasonable criteria below.
good = lambda cat: cat[(cat['snr_grpf'] > 6) & (cat['sigma_m'] > .2)]

#gold sample galaxies 
gold = lambda cat: cat[(cat['ab_mag'] < 25.3)] 

#ambiguity of blends. 
ambig = lambda cat: cat[cat['ambig_blend'] == True ]
not_ambig = lambda cat: cat[cat['ambig_blend'] == False ]
detected = lambda cat: cat[cat['match'] != -1]
not_detected = lambda cat: cat[cat['match'] == -1]

#cuts 
cut_biasiso = lambda cat,bias_cut: cat[(abs(cat['bias_g1']) < bias_cut) & (abs(cat['bias_g2']) < bias_cut)]
cut_biasgrp = lambda cat,bias_cut: cat[(abs(cat['bias_g1_grp']) < bias_cut) & (abs(cat['bias_g2_grp']) < bias_cut)]
down_cut = lambda cat,param,cut: cat[cat[param] < cut]
up_cut = lambda cat,param,cut: cat[cat[param] > cut]
abs_cut = lambda cat,param,cut: cat[abs(cat[param]) < cut]
unphysical_iso = lambda cat: cat[(abs(cat['bias_g1']) > 1.) | (abs(cat['bias_g2']) > 1.)]
unphysical_grp = lambda cat: cat[(abs(cat['bias_g1_grp']) > 1.) | (abs(cat['bias_g2_grp']) > 1.)]

#more specific 
detc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] != -1)]
notdetc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] == -1)]
detc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] != -1)]
notdetc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] == -1)]
best = detc_and_notambig

### extract into image that will be SExtracted 

In [21]:
import descwl
reader = descwl.output.Reader(inputs['input_name'])
results = reader.results
img = results.survey.image

results.add_noise(noise_seed=0) #same noise_seed for all? 
f = fits.PrimaryHDU(results.survey.image.array)
f.writeto(inputs['image_fits'])

### Run source extractor.

In [28]:
#run SExtractor: not sure how to do this using slac shell... 
for i,key in enumerate(run_names):
    if use[i] == True:
        cmd = 'sex {0} -c {1} -CATALOG_NAME {2}'.format(inputs['image_fits_slac'],inputs['config_file'],output_names_slac[key])
        for j,parameter in enumerate(parameter_names):
            cmd = cmd + ' -{0} {1}'.format(parameter, runs[key][j])
        print cmd 
        print

sex /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/image1.fits -c /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/default.sex -CATALOG_NAME /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/out_default.cat -DETECT_THRESH 1.5 -DETECT_MINAREA 5 -FILTER_NAME default.conv

sex /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/image1.fits -c /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/default.sex -CATALOG_NAME /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/out_point_optimized.cat -DETECT_THRESH 5.0 -DETECT_MINAREA 1.0 -FILTER_NAME gauss_3.5_9x9.conv

sex /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/image1.fits -c /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/default.sex -CATALOG_NAME /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sextractor_runs/out_extended_optimized.cat -DETECT_THRESH 1.5 -DETECT_MINAREA 5 -FILTER_NAME gauss_2.0_5x5.conv

sex /nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data/sext

### Create final fits 

In [11]:
width, height = 4500,4500
pixel_scale = .2 

In [16]:
for i,key in enumerate(run_names):
    if use[i] == True:
        #read corresponding catalog 
        cat = descwl.output.Reader(temps['input_name']).results
        table = cat.table
        detected,matched,indices,distance = cat.match_sextractor(output_names[key])

        #convert to arcsecs and relative to image_center 
        detected['X_IMAGE'] = (detected['X_IMAGE'] - 0.5*width - 0.5)*pixel_scale
        detected['Y_IMAGE'] = (detected['Y_IMAGE'] - 0.5*height - 0.5)*pixel_scale

        #convert second moments arcsecs 
        detected['X2_IMAGE']*=pixel_scale**2 
        detected['Y2_IMAGE']*=pixel_scale**2 
        detected['XY_IMAGE']*=pixel_scale**2 

        # calculate size from moments X2_IMAGE,Y2_IMAGE,XY_IMAGE -> remember in pixel**2 so have to convert to arcsecs. 
        sigmas = []
        for x2,y2,xy in zip(detected['X2_IMAGE'],detected['Y2_IMAGE'],detected['XY_IMAGE']):
            second_moments = np.array([[x2,xy],[xy,y2]])
            sigma = np.linalg.det(second_moments)**(+1./4) #should be a plus 
            sigmas.append(sigma)
        SIGMA = Table.Column(name='SIGMA',data=sigmas)
        detected.add_column(SIGMA)

        # #add sizes of the corresponding entries from lsst to the detected table. 
        # SIGMA_M = Table.Column(name='SIGMA_M', data=table[indices]['sigma_m'])
        # detected.add_column(SIGMA_M) 

        ambg_blends = detected_ambiguous_blends(table, indices, detected)
        ambg_blends_indices = list(ambg_blends)
        ambg_blends_ids = list(table[ambg_blends_indices]['db_id'])

        #add columns to table of undetected and ambiguosly blended 
        ambigous_blend_column = []
        for i,gal_row in enumerate(table):
            if i in ambg_blends_indices:
                ambigous_blend_column.append(True)
            else: 
                ambigous_blend_column.append(False)
        column = Table.Column(name='ambig_blend',data=ambigous_blend_column)
        table.add_column(column)

        #create new copy of fits_file with this new table. 
        subprocess.call('cp {0} {1}'.format(temps['input_name'],final_fits_names[key]),shell=True)
        f = fits.open(final_fits_names[key])  
        #replace table and write file
        f[1]= astropy.io.fits.table_to_hdu(table)
        subprocess.call('rm {0}'.format(final_fits_names[key]),shell=True)
        f.writeto(final_fits_names[key]) #overwrites the existing one. 

### Analysis of results, 

In [59]:
cats = collections.OrderedDict()
for key in runs:
    cats[key] = astropy.table.Table.read(fits.open(final_fits_names[key]),hdu=1)

In [64]:
print cats.keys()
print final_fits_names.values()

['default', 'point_optimized', 'extended_optimized', 'extended_optimized_1', 'extended_optimized_2', 'extended_optimized_3']
['/Users/Ismael/temp_data/default_results.fits', '/Users/Ismael/temp_data/point_optimized_results.fits', '/Users/Ismael/temp_data/extended_optimized_results.fits', '/Users/Ismael/temp_data/extended_optimized_1_results.fits', '/Users/Ismael/temp_data/extended_optimized_2_results.fits', '/Users/Ismael/temp_data/extended_optimized_3_results.fits']


In [65]:
table_names = [
'all',
'gold',
'good', 
]

table_functions = [
lambda x: x, 
gold,
good, 
]

column_names = [
'DET_THR',
'DET_MIN_AREA', 
'FILT_SIZE',
'FILT_FWHM(pix)',
'det,!ambig',
'det,ambig', 
'!det,ambig', 
'!det,!ambig',        
]

parameter_table_names = ['DET_THR','DET_MIN_AREA', 'FILT_SIZE','FILT_FWHM(pix)']


run_table_values = collections.OrderedDict()
run_table_values['default'] = [1.5,5,'3x3',2.0]
run_table_values['point_optimized'] = [5.,1.,'9x9',3.5]
run_table_values['extended_optimized'] = [1.5,5.,'5x5',2.]
run_table_values['extended_optimized_1']=[1.5,5.,'9x9',3.5]
run_table_values['extended_optimized_2']=[1.5,1.,'5x5',2.]
run_table_values['extended_optimized_3']=[5.,5.,'5x5',2.]



#first is a place holder so that indices align. 
#this is the function that calculates each column given the catalogue of tha row. 
column_functions = [
None,
None, 
None, 
None,
lambda x: float(len(detc_and_notambig(x)))/len(x)*100,
lambda x: float(len(detc_and_ambig(x)))/len(x)*100,
lambda x: float(len(notdetc_and_ambig(x)))/len(x)*100,
lambda x: float(len(notdetc_and_notambig(x)))/len(x)*100,
]

# row_names = ['{0},({1},{2},{3})'.format(key,value[0],value[1],value[2][:10]) for key,value in runs.iteritems()]

tables =  collections.OrderedDict() # all, good, gold 

for i,table_name in enumerate(table_names): 
    table = collections.OrderedDict()
    table_cats = collections.OrderedDict()
    for name,cat in cats.iteritems():
        table_cats[name] = table_functions[i](cat)
    for j,column_name in enumerate(column_names): 
        if column_name in parameter_table_names:
            table[column_name] = [value[j] for value in run_table_values.values()]
        else:
            table[column_name] = [round(column_functions[j](cat),1) for cat in table_cats.values()]
    tables[table_name] = table

In [71]:
from astropy.table import Table, Column

In [73]:
for name, table in tables.iteritems():
    print name 
    Table(table).show_in_browser(jsviewer=True)
#     Table(tables[i]).show_in_notebook()

all
gold
good
